In [40]:
# Imports
import pandas as pd
import numpy as np
import requests
import os
import json
bikes_df = pd.read_csv('bikes_df.csv')

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [41]:
# First Foursquare API call to retrieve Foursquare IDs for each Point of Interest (POI) and store them in a list
four_square = []
FOUR_SQUARE_KEY = os.environ['FOUR_SQUARE_KEY']

# Iterate through the bike stations DataFrame
for num in range(len(bikes_df)):
    # Construct the Foursquare API URL to search for nearby restaurants, bars, and museums
    url = f"https://api.foursquare.com/v3/places/search?query=restaurant%2Cbar%2Cmuseum&ll={bikes_df['latitude'][num]}%2C{bikes_df['longitude'][num]}&radius=1000&fields=fsq_id%2Cdistance"

    headers = {
        "Accept": "application/json",
        "Authorization": FOUR_SQUARE_KEY
    }
    
    # Make the API request and store the response in the 'four_square' list
    response = requests.get(url, headers=headers)
    call = response.json()
    four_square.append(call)
    
# Normalize the JSON data and create a list of Foursquare IDs
fsq_ids = pd.json_normalize(four_square, record_path='results', meta=[['context','geo_bounds','circle','center','latitude'],
                                                                    ['context','geo_bounds','circle','center','longitude']])
fsq_ids_list = fsq_ids.values.tolist()

# Initialize a list to store Foursquare locales
foursquare_locales = []

# Perform subsequent Foursquare API calls to fetch additional data for each locale
for num in range(0, 1001):
    url = f"https://api.foursquare.com/v3/places/{fsq_ids_list[num][0]}?fields=rating%2Cname%2Clocation%2Cfsq_id%2Cpopularity%2Chours%2Cprice%2Cmenu%2Ctastes%2Cdistance%2Ccategories"
    
    headers = {
        "Accept": "application/json",
        "Authorization": FOUR_SQUARE_KEY
    }

    response = requests.get(url, headers=headers)
    locales = response.json()
    foursquare_locales.append(locales)

# Repeat the process for the next batch of locales
for num in range(1001, 2001):
    url = f"https://api.foursquare.com/v3/places/{fsq_ids_list[num][0]}"
    
    headers = {
        "Accept": "application/json",
        "Authorization": FOUR_SQUARE_KEY
    }

    response = requests.get(url, headers=headers)
    locales = response.json()
    foursquare_locales.append(locales)

# Fetch data for the remaining locales
for num in range(2001, len(fsq_ids_list)):
    url = f"https://api.foursquare.com/v3/places/{fsq_ids_list[num][0]}"
    
    headers = {
        "Accept": "application/json",
        "Authorization": FOUR_SQUARE_KEY
    }

    response = requests.get(url, headers=headers)
    locales = response.json()
    foursquare_locales.append(locales)


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# Normalize the Foursquare locales data to extract information about categories and merge it with other locale details
# Create a DataFrame 'fsq_df' with relevant columns
fsq_df = pd.json_normalize(foursquare_locales, record_path='categories', meta=['fsq_id'])

# Normalize the Foursquare locales data again to extract additional information
fsq_df2 = pd.json_normalize(foursquare_locales)

# Merge the two DataFrames based on the 'fsq_id' column
fsq_df = pd.merge(fsq_df2[['name', 'fsq_id', 'rating', 'popularity', 'hours.display', 'location.address']], fsq_df, on='fsq_id')

# Drop unnecessary columns and rename columns for clarity
fsq_df = fsq_df.drop(columns=['icon.prefix', 'icon.suffix', 'id']).rename(columns={'name_x': 'name', 'popularity': 'popularity(0-1)',
                                                                                'hours.display': 'hours', 'location.address': 'address',
                                                                                'name_y': 'category'})


Put your parsed results into a DataFrame

In [ ]:
# Merge the 'fsq_df' DataFrame with the 'fsq_ids' DataFrame based on the 'fsq_id' column
final_fsq_df = pd.merge(fsq_df, fsq_ids, on='fsq_id')

# Rename specific columns to clarify their meaning
final_fsq_df = final_fsq_df.rename(columns={'context.geo_bounds.circle.center.latitude': 'bike_station_lat',
                                             'context.geo_bounds.circle.center.longitude': 'bike_station_long'})

# Drop any duplicate rows to ensure data integrity
final_fsq_df = final_fsq_df.drop_duplicates()

# Save the final merged DataFrame to a CSV file for future use
final_fsq_df.to_csv('final_fsq_df.csv', index=False)


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [27]:
# Perform Yelp API calls to retrieve information about locales near bike stations
YELP_KEY = os.environ['YELP_KEY']  
yelp_locales = [] 

# Iterate through the bike stations DataFrame
for num in range(len(bikes_df)):
    # Construct the Yelp API URL to search for bars and beaches near each bike station
    yelp_url = f"https://api.yelp.com/v3/businesses/search?latitude={bikes_df['latitude'][num]}&longitude={bikes_df['longitude'][num]}&radius=1000&categories=bars%2Cbeaches&sort_by=best_match&limit=5"
    
    headers = {
        "accept": "application/json",
        "authorization": f'Bearer {YELP_KEY}'  # Include the Yelp API key in the request headers
    }

    # Make the API request and store the response in the 'yelp_locales' list
    response = requests.get(yelp_url, headers=headers)
    yelp_result = response.json()
    yelp_locales.append(yelp_result)


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [30]:
# Normalize Yelp locale data to extract relevant information and create DataFrames
yelp_df1 = pd.json_normalize(yelp_locales, record_path='businesses', meta=[['region','center','latitude'], ['region','center','longitude']])

# Select specific columns from the normalized DataFrame
yelp_df1 = yelp_df1[['id', 'region.center.latitude', 'region.center.longitude']]

# Normalize Yelp locale data again to extract additional information
yelp_df2 = pd.json_normalize(yelp_locales, record_path=['businesses', 'categories'], meta=[['results','id'],['results','name'],
                                    ['results','review_count'],['results','location','address1'],['results','rating']])

# Rename columns
yelp_df2 = yelp_df2.rename(columns={'results.id':'id'})


Put your parsed results into a DataFrame

In [32]:
# Merge the two Yelp DataFrames 'yelp_df1' and 'yelp_df2' based on the 'id' column
final_yelp_df = pd.merge(yelp_df1, yelp_df2, on='id')

# Select specific columns of interest from the merged DataFrame
final_yelp_df = final_yelp_df[['results.name', 
                               'results.location.address1', 
                               'title', 
                               'results.rating', 
                               'results.review_count',
                               'region.center.latitude', 
                               'region.center.longitude']]

# Rename columns for clarity and consistency
final_yelp_df = final_yelp_df.rename(columns={'results.name': 'name', 
                                              'results.location.address1': 'address',
                                              'title': 'category', 
                                              'results.rating': 'yelp_rating',
                                              'results.review_count': 'review_count',
                                              'region.center.latitude': 'bike_station_lat',
                                              'region.center.longitude': 'bike_station_long'})

# Drop duplicate rows to ensure data integrity
final_yelp_df = final_yelp_df.drop_duplicates()

# Display the final Yelp DataFrame
final_yelp_df


,name,address,category,yelp_rating,review_count,bike_station_lat,bike_station_long
0,Bestia,2121 E 7th Pl,Italian,4.5,7473,34.0348,-118.23128
1,Bestia,2121 E 7th Pl,Cocktail Bars,4.5,7473,34.0348,-118.23128
2,Bestia,2121 E 7th Pl,Pizza,4.5,7473,34.0348,-118.23128
15,Bestia,2121 E 7th Pl,Italian,4.5,7473,34.03919,-118.23253
16,Bestia,2121 E 7th Pl,Cocktail Bars,4.5,7473,34.03919,-118.23253
...,...,...,...,...,...,...,...
16197,The Wolves,519 S Spring St,Cocktail Bars,4.0,304,34.04744,-118.24794
16198,The Wolves,519 S Spring St,American (New),4.0,304,34.04744,-118.24794
16199,Mrs Fish,448 South Hill St,Cocktail Bars,4.0,1551,34.04804,-118.25374
16200,Mrs Fish,448 South Hill St,Sushi Bars,4.0,1551,34.04804,-118.25374


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Foursquare provided data for 876 locations, while Yelp returned information for only 365 locations. Foursquare employs a more precise 10-point rating system, whereas Yelp uses a 5-point system. Additionally, Yelp includes a count of the number of reviews for each location. In my opinion, Foursquare offered a more comprehensive view of both the quantity and quality of locales within the vicinity of each bike station. However, it's worth noting that both platforms offer additional metrics that I chose not to utilize for this specific project, as I deemed them unnecessary.

Get the top 10 restaurants according to their rating

In [37]:
# Read the saved CSV files into DataFrames for Yelp and Foursquare data
final_yelp_df = pd.read_csv('final_yelp_df.csv')
final_fsq_df = pd.read_csv('final_fsq_df.csv')

# Filter Foursquare data to include only restaurants
foursquare_restaurant = final_fsq_df[final_fsq_df['category'].str.contains('Restaurant')]

# Group and count the top 10 rated restaurants from Foursquare data
foursquare_top_10 = foursquare_restaurant.groupby(['name', 'rating']).count().sort_values('rating', ascending=False).head(10)

# Group and count the top 10 rated locales from Yelp data
yelp_top_10 = final_yelp_df.groupby(['yelp_name', 'rating']).count().sort_values(by='rating', ascending=False).head(10)


In [43]:
four_square_top_10

,,fsq_id,popularity(0-1),hours,address,category,distance,bike_station_lat,bike_station_long
name,rating,,,,,,,,
Document Coffee Bar,9.2,4,4,4,4,4,4,4,4
Everson Rocye Bar,9.1,6,6,6,6,6,6,6,6
Jitlada Thai Restaurant,8.9,3,3,3,3,3,3,3,3
Bar Amá,8.9,26,26,26,26,26,26,26,26
The Misfit Bar & Restaurant,8.8,1,1,1,1,1,1,1,1
Ruen Pair Thai Restaurant,8.7,3,3,3,3,3,3,3,3
Barrique,8.6,12,12,12,12,12,12,12,12
Versailles Cuban Restaurant,8.6,18,18,18,18,18,18,18,18
The Dresden Restaurant,8.6,9,9,9,9,9,9,9,9


In [42]:
yelp_top_10

,,yelp_address,yelp_category,review_count,bike_station_lat,bike_station_long
yelp_name,rating,,,,,
Drink & Enjoy Premium Bartending,5.0,0,10,10,10,10
HiDef Brewing,5.0,3,3,3,3,3
Olivia,5.0,18,18,18,18,18
Propaganda Wine Bar,5.0,6,6,6,6,6
Ramirez Bartending Co,5.0,0,6,6,6,6
Red Room,5.0,6,6,6,6,6
Rooftop Cinema Club Arts District,5.0,3,3,3,3,3
SoCal Vibes Abbot Kinney,5.0,3,3,3,3,3
Kaos Bartending,5.0,0,4,4,4,4
